<a href="https://colab.research.google.com/github/danielmachlab/imdb_review_classifier/blob/development/imdb_review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB Review Classifier

### Dev Steps
1. Import Dataset from keras/tf
2. View Sentences
3. Translate encoded reviews 
4. Create Model
5. Train Model
6. Test Model
7. Tweak Hyper Parameters


In [58]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb

TensorFlow is already loaded. Please restart the runtime to change versions.


### View Data

In [0]:
num_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [60]:
word_index = imdb.get_word_index()
# shift all numbers by three to make room for placeholders
word_index = {k: (v+3) for k, v in word_index.items()} 

# add placeholders to the word_index dictionary
word_index.update({"<PAD>": 0, "<START>": 1, "<UNK>": 2, "<UNUSED>": 3})
list(word_index.items())[-10:]

[('hardboiled', 52008),
 ("voorhees'", 88586),
 ('35mm', 16818),
 ("'l'", 88587),
 ('paget', 18512),
 ('expands', 20600),
 ('<PAD>', 0),
 ('<START>', 1),
 ('<UNK>', 2),
 ('<UNUSED>', 3)]

In [61]:
# view the first review
print('sentence', x_train[0])
print('label', y_train[1])

sentence [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
label 0


### Translate Encoded Reviews

In [0]:
flipped_word_index = dict([ (v,k) for (k, v) in word_index.items() ])

In [0]:
def translate_review(review):
  return " ".join( [flipped_word_index.get(word, '?') for word in review] )

In [64]:
translate_review(x_train[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [0]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train, value=word_index["<PAD>"], padding="post", maxlen=250)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, value=word_index["<PAD>"], padding="post", maxlen=250)

# Write Models

Word embedding (?)

In [66]:
emb = keras.Sequential()
emb.add(keras.layers.Embedding(10000, 16))
emb.add(keras.layers.GlobalAveragePooling1D())
emb.add(keras.layers.Dense(16, activation="relu"))
emb.add(keras.layers.Dense(1, activation="sigmoid"))

emb.summary()
emb.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
emb.fit(x_train, y_train, epochs=10, batch_size=512, validation_split=0.3)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_4 ( (None, 16)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
35/35 [==============================] - 1s 19ms/step - loss: 0.6921 - accuracy: 0.5573 - val_loss: 0.6896 - val_accuracy: 0.6291
Epoch 2/10
35/35 [==============================] - 1s 17ms/step - loss: 0.6857 - accuracy: 0.7162 - val

LSTM

In [67]:
lstm = keras.Sequential()
lstm.add(keras.layers.Embedding(num_words, 32))
lstm.add(keras.layers.LSTM(32))
lstm.add(keras.layers.Dense(1, activation='sigmoid'))
lstm.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
lstm.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)


Epoch 1/10
157/157 [==============================] - 27s 169ms/step - loss: 0.6735 - acc: 0.5842 - val_loss: 0.6080 - val_acc: 0.7050
Epoch 2/10
157/157 [==============================] - 28s 179ms/step - loss: 0.5826 - acc: 0.7310 - val_loss: 0.4961 - val_acc: 0.8048
Epoch 3/10
157/157 [==============================] - 28s 179ms/step - loss: 0.4536 - acc: 0.8263 - val_loss: 0.4275 - val_acc: 0.8356
Epoch 4/10
157/157 [==============================] - 28s 179ms/step - loss: 0.3797 - acc: 0.8677 - val_loss: 0.7978 - val_acc: 0.6022
Epoch 5/10
157/157 [==============================] - 28s 177ms/step - loss: 0.3569 - acc: 0.8724 - val_loss: 0.7695 - val_acc: 0.7296
Epoch 6/10
157/157 [==============================] - 28s 177ms/step - loss: 0.3173 - acc: 0.8954 - val_loss: 0.5717 - val_acc: 0.7616
Epoch 7/10
157/157 [==============================] - 28s 181ms/step - loss: 0.2736 - acc: 0.9108 - val_loss: 0.5831 - val_acc: 0.8270
Epoch 8/10
157/157 [==============================] - 2

GRU

In [68]:
gru = keras.Sequential()
gru.add(keras.layers.Embedding(num_words, 32))
gru.add(keras.layers.GRU(32))
gru.add(keras.layers.Dense(1, activation='sigmoid'))
gru.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
gru.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 26s 166ms/step - loss: 0.6931 - acc: 0.5235 - val_loss: 0.6713 - val_acc: 0.5688
Epoch 2/10
157/157 [==============================] - 26s 164ms/step - loss: 0.6412 - acc: 0.6348 - val_loss: 0.6003 - val_acc: 0.6466
Epoch 3/10
157/157 [==============================] - 26s 164ms/step - loss: 0.5587 - acc: 0.7249 - val_loss: 0.5405 - val_acc: 0.7160
Epoch 4/10
157/157 [==============================] - 27s 174ms/step - loss: 0.4487 - acc: 0.8256 - val_loss: 0.4974 - val_acc: 0.7864
Epoch 5/10
157/157 [==============================] - 29s 185ms/step - loss: 0.3940 - acc: 0.8577 - val_loss: 0.7389 - val_acc: 0.6182
Epoch 6/10
157/157 [==============================] - 29s 182ms/step - loss: 0.3619 - acc: 0.8726 - val_loss: 0.4554 - val_acc: 0.8310
Epoch 7/10
157/157 [==============================] - 28s 178ms/step - loss: 0.3508 - acc: 0.8788 - val_loss: 0.4254 - val_acc: 0.8416
Epoch 8/10
157/157 [==============================] - 2

Simple RNN

In [69]:
srnn = keras.Sequential()
srnn.add(keras.layers.Embedding(num_words, 32))
srnn.add(keras.layers.SimpleRNN(32))
srnn.add(keras.layers.Dense(1, activation='sigmoid'))
srnn.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
srnn.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 14s 87ms/step - loss: 0.6922 - acc: 0.5063 - val_loss: 0.6854 - val_acc: 0.5346
Epoch 2/10
157/157 [==============================] - 13s 84ms/step - loss: 0.6580 - acc: 0.5993 - val_loss: 0.6628 - val_acc: 0.5892
Epoch 3/10
157/157 [==============================] - 13s 85ms/step - loss: 0.5814 - acc: 0.6938 - val_loss: 0.6626 - val_acc: 0.6296
Epoch 4/10
157/157 [==============================] - 13s 84ms/step - loss: 0.5183 - acc: 0.7607 - val_loss: 0.8173 - val_acc: 0.5650
Epoch 5/10
157/157 [==============================] - 13s 85ms/step - loss: 0.5062 - acc: 0.7617 - val_loss: 0.5864 - val_acc: 0.7116
Epoch 6/10
157/157 [==============================] - 13s 85ms/step - loss: 0.4616 - acc: 0.7871 - val_loss: 0.6120 - val_acc: 0.7220
Epoch 7/10
157/157 [==============================] - 14s 86ms/step - loss: 0.4573 - acc: 0.7808 - val_loss: 0.6232 - val_acc: 0.7026
Epoch 8/10
157/157 [==============================] - 14s 87ms

# Evaluate Models

Word embedding (?)

In [70]:
emb.evaluate(x_test, y_test)

782/782 [==============================] - 1s 2ms/step - loss: 0.3905 - accuracy: 0.8533


[0.3905027210712433, 0.8533200025558472]

LSTM

In [71]:
lstm.evaluate(x_test, y_test)

782/782 [==============================] - 20s 25ms/step - loss: 0.3780 - acc: 0.8567


[0.3780137002468109, 0.8566799759864807]

GRU

In [72]:
gru.evaluate(x_test, y_test)

782/782 [==============================] - 16s 21ms/step - loss: 0.4118 - acc: 0.8456


[0.4117550253868103, 0.8455600142478943]

Simple RNN

In [73]:
srnn.evaluate(x_test, y_test)

782/782 [==============================] - 11s 14ms/step - loss: 0.6009 - acc: 0.7550


[0.600864827632904, 0.7550399899482727]